In [167]:
import spotipy
import spotipy.util as util
import pickle
import sys
import requests
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import string
import json
import unidecode
import sys
!{sys.executable} -m pip install pytrends
import time
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from pytrends.request import TrendReq


In [168]:
#Read song data frame from pkl file, needs to change it in step 1
with open('song_df.pkl', 'rb') as f:
        song_df = pickle.load(f)

In [169]:
song_df

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.7510,0.00000,0.1050,0.4460,109.891,4.0,5658aM19fA3JVwTK6eQX70,2019-05-17,day,Vertigo Berlin
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.1920,0.00244,0.0863,0.5530,120.042,4.0,4g1ZRSobMefqF6nelkgibi,2019-09-06,day,Republic Records
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,0TK2YIli7K1leLovkQiNik,False,190960.0,1.0,1.0,...,0.0370,0.00000,0.0945,0.7500,116.947,4.0,2ZaX1FdZCwchXl1QZiD4O4,2019-06-21,day,Island Records
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.1100,0.00000,0.1230,0.4120,158.087,4.0,74gSdSHe71q7urGWMMn3qB,2019-05-03,day,Nice Life/Atlantic
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.5760,0.00000,0.2100,0.0916,101.993,4.0,3tBkjgxDqAwss76O1YHsSY,2019-10-23,day,Interscope Records
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113178,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,0.0105,0.03410,0.4030,0.7020,124.016,4.0,3sr6lAuO3nmB1u8ZuQgpiX,1997-01-01,day,Virgin Records
113192,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,0.4320,0.00168,0.0446,0.5450,79.913,4.0,6qbvgcaWgMn2BQY6FhdOm2,2018-11-16,day,Warner Music Group - X5 Music Group
113194,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,0RlWNvJKKp0gODH7sUwm1j,False,275493.0,1.0,7.0,...,0.0579,0.82900,0.0561,0.7230,97.518,4.0,0Av1HIrrBAtIbQTaAqo8Lb,2011-11-07,day,2011 Ameritz Music Ltd
113198,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,6xAF3xYDK915NEvC6WNbj9,False,214240.0,1.0,4.0,...,0.2610,0.00000,0.5480,0.9030,122.025,4.0,4UMe0Ods7kygK6OISasZe9,1997-11-04,day,Mercury Nashville


In [170]:
#Drop lines containing N/A
song_dropna = song_df.dropna().copy()
song_dropna

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.7510,0.00000,0.1050,0.4460,109.891,4.0,5658aM19fA3JVwTK6eQX70,2019-05-17,day,Vertigo Berlin
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.1920,0.00244,0.0863,0.5530,120.042,4.0,4g1ZRSobMefqF6nelkgibi,2019-09-06,day,Republic Records
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,0TK2YIli7K1leLovkQiNik,False,190960.0,1.0,1.0,...,0.0370,0.00000,0.0945,0.7500,116.947,4.0,2ZaX1FdZCwchXl1QZiD4O4,2019-06-21,day,Island Records
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.1100,0.00000,0.1230,0.4120,158.087,4.0,74gSdSHe71q7urGWMMn3qB,2019-05-03,day,Nice Life/Atlantic
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.5760,0.00000,0.2100,0.0916,101.993,4.0,3tBkjgxDqAwss76O1YHsSY,2019-10-23,day,Interscope Records
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113178,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,0.0105,0.03410,0.4030,0.7020,124.016,4.0,3sr6lAuO3nmB1u8ZuQgpiX,1997-01-01,day,Virgin Records
113192,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,0.4320,0.00168,0.0446,0.5450,79.913,4.0,6qbvgcaWgMn2BQY6FhdOm2,2018-11-16,day,Warner Music Group - X5 Music Group
113194,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,0RlWNvJKKp0gODH7sUwm1j,False,275493.0,1.0,7.0,...,0.0579,0.82900,0.0561,0.7230,97.518,4.0,0Av1HIrrBAtIbQTaAqo8Lb,2011-11-07,day,2011 Ameritz Music Ltd
113198,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,6xAF3xYDK915NEvC6WNbj9,False,214240.0,1.0,4.0,...,0.2610,0.00000,0.5480,0.9030,122.025,4.0,4UMe0Ods7kygK6OISasZe9,1997-11-04,day,Mercury Nashville


In [171]:
from pytrends.request import TrendReq
from random import randint

In [172]:
#this function returns the mean trend of the past n months of a song in song_df 
def google_trend(song_df, song, n):
    trend_date = song_df.iloc[song]["spotify_album_release_date"]
    
    if len(trend_date) > 5:
        dt = datetime.strptime(trend_date, '%Y-%m-%d')
    else:
        dt = datetime.strptime(trend_date, '%Y')
    
    dt_start = dt + relativedelta(months=-n)
   
    trend_artist = song_df.iloc[song]["artist"]
    kw_list = [trend_artist]
    
    #set dataframe, the earliset is 1/1/2004
    earliest_date = '2004-1-1'
    google_earliest = datetime.strptime(earliest_date, '%Y-%m-%d')
    if dt_start >= google_earliest:
        pytrends = TrendReq(hl='en-US', tz=360)
        time_period = dt_start.strftime("%Y") + '-' + dt_start.strftime("%m") + '-' + dt_start.strftime("%d") + ' ' + dt.strftime("%Y") + '-' + dt.strftime("%m") + '-' + dt.strftime("%d")
        pytrends.build_payload(kw_list, cat=0, timeframe= time_period, geo='', gprop='')
        trend = pytrends.interest_over_time()
        if not trend.empty:
            trend_mean=np.mean(trend.iloc[:,0])
            return trend_mean
        else:
            return 0
    elif dt_start < google_earliest and dt > google_earliest:
        pytrends = TrendReq(hl='en-US', tz=360)
        time_period = google_earliest.strftime("%Y") + '-' + google_earliest.strftime("%m") + '-' + google_earliest.strftime("%d") + ' ' + dt.strftime("%Y") + '-' + dt.strftime("%m") + '-' + dt.strftime("%d")
        pytrends.build_payload(kw_list, cat=0, timeframe= time_period, geo='', gprop='')
        trend = pytrends.interest_over_time()
        if not trend.empty:
            trend_mean=np.mean(trend.iloc[:,0])
            return trend_mean
        else:
            return 0
    elif dt <= google_earliest:
        trend_mean = -1
        return trend_mean

#test    
print(google_trend(song_df, 0, 3))

36.22222222222222


In [173]:
# fill in trend feature from mth row to nth row
def trend_df(google_trend_df, song_dropna, m, n):

    i=0
    for song in range(m, n, 1):
        
        try: 
            google_trend_df = google_trend_df.append({'artist': song_dropna.iloc[song]['artist'], 'title': song_dropna.iloc[song]['title'], 'spotify_id':song_dropna.iloc[song]['spotify_id'],'google_trend': google_trend(song_dropna, song, 3)} , ignore_index=True)
        except requests.exceptions.Timeout:
            print("Timeout occurred")
        time.sleep(randint(3,10))

        print("song_df row No.", end =" ")
        print(song)
        print("Mean trend: ", end =" ")
        print(google_trend_df.iloc[i+1]["google_trend"])
        i=i+1
    
    #save as pkl file
    with open('google_trend_df'+'_'+str(m)+'_'+str(n)+'.pkl', 'wb') as f:
        pickle.dump(google_trend_df, f)

In [183]:
#trend for the last 18 rows
#data = {'artist':[''], 'title':[''], 'spotify_id':[''], 'google_trend':['']} 
#google_trend_df = pd.DataFrame(data)
#trend_df(google_trend_df, song_dropna, 4951, len(song_dropna))

In [ ]:
#check the file is saved correctly
with open('google_trend_df_4951_4969.pkl', 'rb') as f:
        google_trend_df = pickle.load(f)
google_trend_df

trend_feature(song_dropna, 0, 1000)

In [181]:
##Above is the program
#Belove is how to use the program

In [182]:
#To get data for the other rows
# Stpe 1 change the picklefile to read at the beginning of this program
# Stpe 2 Change the parameters in the fuction below
# Stpe 3 Click run
# Step 4(optional) check the file as shown above

#First 1000 rows (from 0 to 1000), saved as song_dropna_0_1000.pkl 
#I used the old data set which has 4969 rows arfer dropping N/A.
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 8001, len(song_dropna))

song_df row No. 8001
Mean trend:  -1
song_df row No. 8002
Mean trend:  40.57142857142857
song_df row No. 8003
Mean trend:  -1
song_df row No. 8004
Mean trend:  0
song_df row No. 8005
Mean trend:  -1
song_df row No. 8006
Mean trend:  -1
song_df row No. 8007
Mean trend:  -1
song_df row No. 8008
Mean trend:  -1
song_df row No. 8009
Mean trend:  25.989247311827956
song_df row No. 8010
Mean trend:  -1
song_df row No. 8011
Mean trend:  -1
song_df row No. 8012
Mean trend:  -1
song_df row No. 8013
Mean trend:  -1
song_df row No. 8014
Mean trend:  -1
song_df row No. 8015
Mean trend:  -1
song_df row No. 8016
Mean trend:  -1
song_df row No. 8017
Mean trend:  -1
song_df row No. 8018
Mean trend:  -1
song_df row No. 8019
Mean trend:  -1
song_df row No. 8020
Mean trend:  -1
song_df row No. 8021
Mean trend:  -1
song_df row No. 8022
Mean trend:  -1
song_df row No. 8023
Mean trend:  -1
song_df row No. 8024
Mean trend:  -1
song_df row No. 8025
Mean trend:  -1
song_df row No. 8026
Mean trend:  0
song_df r

In [184]:
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 7001, 8000)

song_df row No. 7001
Mean trend:  -1
song_df row No. 7002
Mean trend:  -1
song_df row No. 7003
Mean trend:  0
song_df row No. 7004
Mean trend:  -1
song_df row No. 7005
Mean trend:  -1
song_df row No. 7006
Mean trend:  0
song_df row No. 7007
Mean trend:  -1
song_df row No. 7008
Mean trend:  0
song_df row No. 7009
Mean trend:  -1
song_df row No. 7010
Mean trend:  -1
song_df row No. 7011
Mean trend:  25.182795698924732
song_df row No. 7012
Mean trend:  -1
song_df row No. 7013
Mean trend:  -1
song_df row No. 7014
Mean trend:  -1
song_df row No. 7015
Mean trend:  18.311827956989248
song_df row No. 7016
Mean trend:  -1
song_df row No. 7017
Mean trend:  7.311827956989247
song_df row No. 7018
Mean trend:  -1
song_df row No. 7019
Mean trend:  -1
song_df row No. 7020
Mean trend:  -1
song_df row No. 7021
Mean trend:  -1
song_df row No. 7022
Mean trend:  -1
song_df row No. 7023
Mean trend:  -1
song_df row No. 7024
Mean trend:  -1
song_df row No. 7025
Mean trend:  12.25
song_df row No. 7026
Mean tr

In [185]:
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 6001, 7000)

song_df row No. 6001
Mean trend:  33.02150537634409
song_df row No. 6002
Mean trend:  2.903225806451613
song_df row No. 6003
Mean trend:  40.70652173913044
song_df row No. 6004
Mean trend:  36.22222222222222
song_df row No. 6005
Mean trend:  36.526881720430104
song_df row No. 6006
Mean trend:  59.39784946236559
song_df row No. 6007
Mean trend:  0
song_df row No. 6008
Mean trend:  -1
song_df row No. 6009
Mean trend:  39.44565217391305
song_df row No. 6010
Mean trend:  48.34444444444444
song_df row No. 6011
Mean trend:  22.43010752688172
song_df row No. 6012
Mean trend:  36.225806451612904
song_df row No. 6013
Mean trend:  34.29032258064516
song_df row No. 6014
Mean trend:  16.236559139784948
song_df row No. 6015
Mean trend:  67.75268817204301
song_df row No. 6016
Mean trend:  0
song_df row No. 6017
Mean trend:  27.010752688172044
song_df row No. 6018
Mean trend:  59.193548387096776
song_df row No. 6019
Mean trend:  19.634408602150536
song_df row No. 6020
Mean trend:  29.666666666666668


In [186]:
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 5001, 6000)

song_df row No. 5001
Mean trend:  0
song_df row No. 5002
Mean trend:  57.391304347826086
song_df row No. 5003
Mean trend:  6.279569892473118
song_df row No. 5004
Mean trend:  44.34782608695652
song_df row No. 5005
Mean trend:  32.365591397849464
song_df row No. 5006
Mean trend:  54.02150537634409
song_df row No. 5007
Mean trend:  25.225806451612904
song_df row No. 5008
Mean trend:  47.204301075268816
song_df row No. 5009
Mean trend:  60.526881720430104
song_df row No. 5010
Mean trend:  0
song_df row No. 5011
Mean trend:  44.56989247311828
song_df row No. 5012
Mean trend:  37.17204301075269
song_df row No. 5013
Mean trend:  1.075268817204301
song_df row No. 5014
Mean trend:  54.02150537634409
song_df row No. 5015
Mean trend:  0
song_df row No. 5016
Mean trend:  0
song_df row No. 5017
Mean trend:  44.41935483870968
song_df row No. 5018
Mean trend:  33.67741935483871
song_df row No. 5019
Mean trend:  25.010752688172044
song_df row No. 5020
Mean trend:  57.34782608695652
song_df row No. 50

ResponseError: The request failed: Google returned a response with code 429.

In [ ]:
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 4000, 5000)

In [ ]:
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 3000, 4000)

In [ ]:
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 2000, 3000)

In [ ]:
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 1000, 2000)

In [ ]:
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 0, 1000)